From the image of the montly histories, produce feature collections and, ultimately, CSV with their reduction over city outlines.

In [9]:
import ee
import geemap
import geetools
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
ee.Initialize()

In [4]:
# Load the cities feature collection and filter for South American countries
southAmericaISO = ['ARG', 'BOL', 'BRA', 'CHL', 'COL', 'ECU', 'GUY', 'PRY', 'PER', 'SUR', 'URY', 'VEN'];
cities = ee.FeatureCollection("projects/dw-city-tree-coverage/assets/city_outlines")\
              .filter(ee.Filter.inList('CTR_MN_ISO', southAmericaISO))\
              .filter(ee.Filter.gte('P15', 7e4));


In [6]:
# For each available month history, access the respective data and reduce it to a region.
# Then, cast to pandas
for month in range(1, 13):
    month_history = ee.Image(f"projects/ee-tree-coverage-cities/assets/pm2p5-monthly-2017-2023/Month_{month}_Mean")
    
    city_month_history = month_history.reduceRegions(
        collection=cities,                  # The cities feature collection
        reducer=ee.Reducer.mean(),          # Use mean reducer
        scale=44528,                        # Adjust scale based on the dataset resolution
    ).map(lambda feature: feature.set({
        'month': month_history.get('month'),
        'lat': feature.geometry().centroid().coordinates().get(1),  # Extract latitude
        'lon': feature.geometry().centroid().coordinates().get(0),  # Extract longitude
    }))
    
    city_month_history = geemap.ee_to_df(city_month_history)
        
    city_month_history.to_csv(f"../output/city-monthly-means/City_Month_{month}_Mean", index=False)

In [7]:
city_month_history

,CTR_MN_ISO,CTR_MN_NM,ID_HDC_G0,P15,UC_NM_LST,UC_NM_MN,lat,lon,mean,month
0,ARG,Argentina,962,441985.894928,San Juan,San Juan,-31.536676,-68.542058,4.678136e-09,12
1,ARG,Argentina,966,937975.566467,Mendoza,Mendoza,-32.906274,-68.814158,5.479471e-09,12
2,ARG,Argentina,967,222796.598145,San Salvador de Jujuy,San Salvador de Jujuy,-24.190622,-65.295457,2.454059e-09,12
3,ARG,Argentina,969,506599.243210,Salta,Salta,-24.807302,-65.415474,4.222186e-09,12
4,ARG,Argentina,974,188613.244995,La Rioja,La Rioja,-29.417105,-66.856459,3.687934e-09,12
...,...,...,...,...,...,...,...,...,...,...
518,VEN,Venezuela,954,129270.409840,Carupano,Carupano,10.651785,-63.257101,5.701253e-09,12
519,VEN,Venezuela,957,654228.726257,Ciudad Guayana; Puerto Ordaz,Ciudad Guayana,8.283047,-62.739250,5.932471e-09,12
520,VEN,Venezuela,960,80444.213928,San Felix,San Felix,8.313809,-62.629488,5.932471e-09,12
521,VEN,Venezuela,963,93348.082642,Upata,Upata,8.008475,-62.398326,NaN,12
